<a href="https://colab.research.google.com/github/jmhuer/Fall2021MusicResearch/blob/main/HumanFeedbackAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import re
import subprocess


def download_gdrive(id, print_stout=True):
  coomand = 'gdown https://drive.google.com/uc?id={}'.format(id)
  returned_value = subprocess.run(coomand, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
  if print_stout: print(returned_value.stdout.decode("utf-8"))
  else: print("Download Complete")

In [2]:
download_gdrive("1TMo-SiXUvMDUSIKPRIhpD1zrp_6gOw44", print_stout=True)

Downloading...
From: https://drive.google.com/uc?id=1TMo-SiXUvMDUSIKPRIhpD1zrp_6gOw44
To: /content/responses.zip
100%|██████████| 480k/480k [00:00<00:00, 4.19MB/s]



In [3]:
!unzip responses.zip
!pip install tika


Archive:  responses.zip
   creating: responses/
  inflating: responses/website email-8.pdf  
  inflating: __MACOSX/responses/._website email-8.pdf  
  inflating: responses/website email-9.pdf  
  inflating: __MACOSX/responses/._website email-9.pdf  
  inflating: responses/website email-16.pdf  
  inflating: __MACOSX/responses/._website email-16.pdf  
  inflating: responses/website email-17.pdf  
  inflating: __MACOSX/responses/._website email-17.pdf  
  inflating: responses/website email-15.pdf  
  inflating: __MACOSX/responses/._website email-15.pdf  
  inflating: responses/website email-14.pdf  
  inflating: __MACOSX/responses/._website email-14.pdf  
  inflating: responses/website email-10.pdf  
  inflating: __MACOSX/responses/._website email-10.pdf  
  inflating: responses/website email-11.pdf  
  inflating: __MACOSX/responses/._website email-11.pdf  
  inflating: responses/website email-13.pdf  
  inflating: __MACOSX/responses/._website email-13.pdf  
  inflating: responses/websit

In [6]:
#!/usr/bin/python3

import os, glob
from tika import parser 
from pandas import DataFrame
import pandas as pd

# What file extension to find, and where to look from
ext = "*.pdf"
PATH = "responses"

# Find all the files with that extension
files = []
for dirpath, dirnames, filenames in os.walk(PATH):
    files += glob.glob(os.path.join(dirpath, ext))

# Create a Pandas Dataframe to hold the filenames and the text
df = DataFrame(columns=("filename","text"))

# Process each file in turn, parsing with Tika and storing in the dataframe
for idx, filename in enumerate(files):
   data = parser.from_file(filename)
   text = data["content"]
   df.loc[idx] = [filename, text]

# For debugging, print what we found
# print(df.loc[0][1])
# indx = df.loc[0][1]



for index, row in df.iterrows():
    indx = row["text"].find("FROM") + len("FROM")
    if indx - len("FROM")!=-1:
      group_number = row["text"][indx:indx+1]
      print(row["filename"], group_number)
    # if group_number==" ":
    #     print(row["text"])


responses/website email-3.pdf 3
responses/website email-10.pdf 1
responses/website email-16.pdf 2
responses/website email-20.pdf 1
responses/website email-15.pdf 2
responses/website email.pdf 2
responses/website email-14.pdf 2
responses/website email-4.pdf 2
responses/website email-17.pdf 3
responses/website email-5.pdf 2
responses/website email-9.pdf 1
responses/website email-21.pdf 3
responses/website email-7.pdf 1
responses/website email-2.pdf 3
responses/website email-11.pdf 1
responses/website email-22.pdf 2
responses/website email-6.pdf 2
responses/website email-13.pdf 3
responses/website email-19.pdf 3
responses/website email-12.pdf 1
responses/website email-8.pdf 1
responses/website email-18.pdf 2


In [102]:
import re
  
# Function to extract all the numbers from the given string
def getNumbers(str):
    array = re.findall(r'[0-9]+', str)
    return array

class SurveyData:
  def __init__(self, raw_dataframe):
      self.raw_dataframe = raw_dataframe
      self.group_map = {1: ["1.0","1.5","0.5","0.7","1.9", 
                            "1.9","1.5","1.0","0.7","0.5", 
                            "0.5","1.0", "1.0", "1.9", "1.5", "1.0", 
                            "2.5","Human","2.5","Human"],
                        2: ["1.0","1.9","0.5","1.5","0.7", 
                            "0.5","0.7","1.0","1.5","1.9", 
                            "0.5","1.0","1.0","1.9","1.5","1.0" 
                            "1.0","Human","1.0","Human"],
                        3: ["0.7","0.5","1.9","1.5","1.0", 
                            "1.9","0.7","1.5","1.0","0.5", 
                            "1.0","1.9", "1.5","1.0","1.5","1.0",
                            "1.0","Human","1.0","Human"]
                        }
      self.indx2Q_map =  {"Q1_": [i for i in range(0,6)],
                          "Q2_": [i for i in range(6,11)],
                          "Q3_": [i for i in range(11,16)],
                          "Q4_": [i for i in range(16,21)]}
      self.d = {'Q1_0.5':[], 'Q1_0.7':[], 'Q1_1.0':[], 'Q1_1.5':[],'Q1_1.9':[], 
                'Q2_0.5':[], 'Q2_0.7':[],' Q2_1.0':[], 'Q2_1.5':[],'Q2_1.9':[],
                'Q3_0.5':[], 'Q3_0.7':[], 'Q3_1.0':[], 'Q3_1.5':[],'Q3_1.9':[],
                'Q4_Human':[],'Q4_1.5':[]}
      self.cleaned_data_dic = self.make_cleaned_data_dic()
  def indx2Q(self, seqIndx):
      # print(seqIndx)
      for Q in self.indx2Q_map.keys():
          if seqIndx in self.indx2Q_map[Q]: 
              # print(Q)
              return Q
  def indx2example(self, seqIndx, group):
      # print(self.group_map[group])
      for ex in self.group_map[group]:
          return self.group_map[group][seqIndx]
  def number2score(self, number, seqIndx):
      if seqIndx < 10: 
        return int(number) - seqIndx*5 
      else: 
        return 1
  def make_cleaned_data_dic(self):
      for index, row in self.raw_dataframe.iterrows():
          indx = row["text"].find("FROM") + len("FROM")
          if indx - len("FROM")!=-1:
              group_number = int(row["text"][indx:indx+1])
          ##DO STUFF 
          vals = []
          # print(getNumbers(row["text"])[5:])
          raw = getNumbers(row["text"])[5:]
          for seqIndx in range(len(raw)):
              c = self.indx2Q(seqIndx) + self.indx2example(seqIndx, group_number)
              print(c)
              print(self.number2score(raw[seqIndx], seqIndx))
              # self.d[c].append(self.number2score(raw[seqIndx], seqIndx))
          
          # ##add a 0 for every row not appended
          # for Q in self.d.keys():
          #     if len(self.d[Q])!=index: 
          #         self.d[Q].append(0)

          df = pd.DataFrame(data=self.d)
      return df

In [103]:
s = SurveyData(df)

Q1_0.7
3
Q1_0.5
-3
Q1_1.9
-4
Q1_1.5
0
Q1_1.0
-1
Q1_1.9
-2
Q2_0.7
0
Q2_1.5
-3
Q2_1.0
-1
Q2_0.5
-2
Q2_1.0
1
Q3_1.9
1
Q3_1.5
1
Q3_1.0
1
Q3_1.5
1
Q3_1.0
1
Q1_1.0
1
Q1_1.5
-3
Q1_0.5
-3
Q1_0.7
-2
Q1_1.9
-3
Q1_1.9
-2
Q2_1.5
-3
Q2_1.0
-3
Q2_0.7
-1
Q2_0.5
-3
Q2_0.5
1
Q3_1.0
1
Q3_1.0
1
Q3_1.9
1
Q3_1.5
1
Q3_1.0
1
Q1_1.0
2
Q1_1.9
-2
Q1_0.5
-4
Q1_1.5
-4
Q1_0.7
-1
Q1_0.5
-3
Q2_0.7
-4
Q2_1.0
-3
Q2_1.5
-2
Q2_1.9
-1
Q2_0.5
1
Q3_1.0
1
Q3_1.0
1
Q3_1.9
1
Q3_1.5
1
Q3_1.01.0
1
Q1_1.0
1
Q1_1.5
-4
Q1_0.5
-3
Q1_0.7
-3
Q1_1.9
-2
Q1_1.9
-1
Q2_1.5
-2
Q2_1.0
-1
Q2_0.7
-2
Q2_0.5
-3
Q2_0.5
1
Q3_1.0
1
Q3_1.0
1
Q3_1.9
1
Q3_1.5
1
Q3_1.0
1
Q1_1.0
2
Q1_1.9
0
Q1_0.5
-2
Q1_1.5
-3
Q1_0.7
-1
Q1_0.5
-2
Q2_0.7
-2
Q2_1.0
-2
Q2_1.5
-1
Q2_1.9
0
Q2_0.5
1
Q3_1.0
1
Q3_1.0
1
Q3_1.9
1
Q3_1.5
1
Q3_1.01.0
1
Q1_1.0
2
Q1_1.9
-1
Q1_0.5
-3
Q1_1.5
-2
Q1_0.7
-2
Q1_0.5
-2
Q2_0.7
-3
Q2_1.0
-2
Q2_1.5
-1
Q2_1.9
-1
Q2_0.5
1
Q3_1.0
1
Q3_1.0
1
Q3_1.9
1
Q3_1.5
1
Q3_1.01.0
1
Q1_1.0
2
Q1_1.9
7
Q1_0.5
-5
Q1_1.5
-7
Q1_0.7
-9
Q1_0.5
-6
Q2_0.7
-8
Q2_1.0
-9